In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("diabetic_data.csv")

In [6]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [7]:
1. #Melt

drug_columns = ['metformin', 'insulin', 'glipizide', 'glyburide', 'pioglitazone']
df_melted = pd.melt(df, id_vars=['encounter_id', 'patient_nbr', 'age', 'gender'],
                    value_vars=drug_columns, var_name='Drug', value_name='DrugResponse')
print(df_melted.head())

   encounter_id  patient_nbr      age  gender       Drug DrugResponse
0       2278392      8222157   [0-10)  Female  metformin           No
1        149190     55629189  [10-20)  Female  metformin           No
2         64410     86047875  [20-30)  Female  metformin           No
3        500364     82442376  [30-40)    Male  metformin           No
4         16680     42519267  [40-50)    Male  metformin           No


In [8]:
melt_columns = ['admission_type_id', 'discharge_disposition_id', 'admission_source_id']
df_adm_melted = pd.melt(
    df,
    id_vars=['encounter_id', 'patient_nbr', 'age', 'gender'],
    value_vars=melt_columns,
    var_name='Admission_Discharge_Type',
    value_name='Type_ID'
)
print(df_adm_melted.head())

   encounter_id  patient_nbr      age  gender Admission_Discharge_Type  \
0       2278392      8222157   [0-10)  Female        admission_type_id   
1        149190     55629189  [10-20)  Female        admission_type_id   
2         64410     86047875  [20-30)  Female        admission_type_id   
3        500364     82442376  [30-40)    Male        admission_type_id   
4         16680     42519267  [40-50)    Male        admission_type_id   

   Type_ID  
0        6  
1        1  
2        1  
3        1  
4        1  


In [12]:
#2. pivot

df_pivoted = df_melted.pivot(index=['encounter_id', 'patient_nbr'], columns='Drug', values='DrugResponse')
print(df_pivoted.head())

Drug                     glipizide glyburide insulin metformin pioglitazone
encounter_id patient_nbr                                                   
12522        48330783       Steady        No  Steady        No           No
15738        63555939           No        No  Steady        No           No
16680        42519267       Steady        No  Steady        No           No
28236        89869032           No        No  Steady        No           No
35754        82637451           No        No  Steady        No           No


In [13]:
df_pivot_adm = df_adm_melted.pivot(
    index=['encounter_id', 'patient_nbr'],
    columns='Admission_Discharge_Type',
    values='Type_ID'
).reset_index()

print(df_pivot_adm)


Admission_Discharge_Type  encounter_id  patient_nbr  admission_source_id  \
0                                12522     48330783                    4   
1                                15738     63555939                    4   
2                                16680     42519267                    7   
3                                28236     89869032                    7   
4                                35754     82637451                    2   
...                                ...          ...                  ...   
101761                       443847548    100162476                    7   
101762                       443847782     74694222                    5   
101763                       443854148     41088789                    7   
101764                       443857166     31693671                    7   
101765                       443867222    175429310                    7   

Admission_Discharge_Type  admission_type_id  discharge_disposition_id  
0              

In [14]:
#3.Aggregation
agg_data = df.agg({
    'time_in_hospital': 'mean',
    'number_diagnoses': 'mean'
})

print(agg_data)

time_in_hospital    4.395987
number_diagnoses    7.422607
dtype: float64


In [15]:
agg_data1 = df.agg({
    'time_in_hospital': 'mean',
    'num_medications': 'mean'
})
print(agg_data1)

time_in_hospital     4.395987
num_medications     16.021844
dtype: float64


In [19]:
#4. Iterations

for index, row in df.head().iterrows():
    print(f"{row['encounter_id']}, Age: {row['age']}, Time in Hospital: {row['time_in_hospital']}")

2278392, Age: [0-10), Time in Hospital: 1
149190, Age: [10-20), Time in Hospital: 3
64410, Age: [20-30), Time in Hospital: 2
500364, Age: [30-40), Time in Hospital: 2
16680, Age: [40-50), Time in Hospital: 1


In [18]:
for index, row in df.head().iterrows():
    print(f"{row['patient_nbr']}, Discharge Disposition ID: {row['discharge_disposition_id']}")

8222157, Discharge Disposition ID: 25
55629189, Discharge Disposition ID: 1
86047875, Discharge Disposition ID: 1
82442376, Discharge Disposition ID: 1
42519267, Discharge Disposition ID: 1


In [21]:
#5. Group by 

df_grouped = df.groupby(['age']).agg({
    'time_in_hospital': 'mean',
    'num_procedures': 'mean'
}).reset_index()
print(df_grouped)

        age  time_in_hospital  num_procedures
0    [0-10)          2.546584        0.180124
1   [10-20)          3.191027        0.335745
2   [20-30)          3.564876        0.767049
3   [30-40)          3.799735        1.113377
4   [40-50)          4.039649        1.341662
5   [50-60)          4.125753        1.556096
6   [60-70)          4.382244        1.573367
7   [70-80)          4.590878        1.377896
8   [80-90)          4.808629        1.023027
9  [90-100)          4.755818        0.670247


In [22]:
df_grp = df.groupby(['gender']).agg({
    'time_in_hospital': 'mean',
    'num_medications': 'mean'
}).reset_index()

print(df_grp)

            gender  time_in_hospital  num_medications
0           Female          4.478797        16.187888
1             Male          4.299777        15.828775
2  Unknown/Invalid          3.333333        16.333333
